In [1]:
import pandas as pd
import names as n
import pipeline_manager

In [2]:
filename = 'data/ga_hits.csv'
ga_hits_df = pd.read_csv(filename, low_memory=False)

filename = 'data/ga_sessions.csv'
ga_sessions_df = pd.read_csv(filename, low_memory=False)

df = pd.merge(ga_sessions_df, ga_hits_df, how="inner", on='session_id')

In [3]:
df =  pipeline_manager.start_pl.fit_transform(df)
df.head(1)

,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,is_target
12337081,2021-06-24,15:00:00,75.0,jaSOmLICuBzCFqHfBdRg,email,LliRUcMuIXWdLyWHGyiO,sDWYAbLNiGZVxGBDdTxc,null,desktop,null,null,null,1280x720,Chrome,Russia,Moscow,1


In [4]:
X = df.drop(columns=n.c.IS_TARGET)
y = df[n.c.IS_TARGET]


X_result = pipeline_manager.main_pl.fit_transform(X, y)

X_result = pipeline_manager.cleaning_outliers_pl.fit_transform(X_result, y)
# After cleaning we need to remove y values with outliers
y_result = y[X_result.index]

X_result = pipeline_manager.choose_best_features_pl.fit_transform(X_result, y_result)

pipeline_manager.best_model_pl.fit(X_result, y_result)

#print(X_result.shape, y_result.shape)
X_result.head(3)

C:\Users\misha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,device_category_desktop,device_category_mobile,device_category_tablet,device_os_(not set),device_os_Android,device_os_Linux,device_os_Windows,device_os_iOS,device_browser_Mozilla Compatible Agent,device_browser_Safari (in-app),...,geo_city_customised_Penza,geo_city_customised_Prineville,geo_city_customised_Pyatigorsk,geo_city_customised_Sochi,geo_city_customised_Stavropol,geo_city_customised_Tomsk,geo_city_customised_Tyumen,geo_city_customised_Vladimir,geo_city_customised_Volgograd,geo_city_customised_Voronezh
12337081,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3288869,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10566794,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = df.drop(columns=n.c.IS_TARGET)
y = df[n.c.IS_TARGET]

In [6]:
border_for_prediction_1 =  0.1

y_predict_proba_result = pipeline_manager.result_pl.predict_proba(X.sample(20000))
y_predict_proba_result = y_predict_proba_result[:, 1]

y_predict_result = [1 if value >= border_for_prediction_1 else 0 for value in y_predict_proba_result]
sum(y_predict_result)

52

In [9]:
import joblib

df.sample(20).to_json('data/random_customers.json', orient='records')
joblib.dump(pipeline_manager.result_pl, 'data/main_pipeline')

['data/main_pipeline']